In [36]:
from keras.models import Model
from keras.layers import Conv2D, Embedding, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Reshape, Activation ,concatenate, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.layers.wrappers import TimeDistributed, Bidirectional

from keras import optimizers
import tensorflow as tf
from keras.layers.recurrent import LSTM
from lwlrap import tf_lwlrap

import numpy as np

In [40]:
class LSTM_attention():
    def __init__(self, num_freq = 128, len_div = 256, num_hidden=300):
        self.num_freq = num_freq
        self.len_div = len_div
        self.num_hidden = num_hidden
        
    def LSTM(self):

        self.inputs = Input(shape=(self.num_freq, self.len_div), name='input')

        self.lstm = LSTM(self.num_hidden, return_sequences=False, name='LSTM')(self.inputs)
        self.drop = Dropout(rate=0.05)(self.lstm)
        self.dense = Dense(80, name='dense1')(self.drop)
        self.pred = Activation('softmax',name='pred')(self.dense)

        adam = optimizers.Adam(lr=0.0001)
        model = Model(inputs=self.inputs, outputs=self.pred)
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[tf_lwlrap])
        return model
    
    def Embed_LSTM(self):
#         adam = optimizers.Adam(lr=0.0001)
        model = Model(inputs=self.inputs, outputs=self.lstm)
#         model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[tf_lwlrap])
        return model

In [41]:
lstm_attn = LSTM_attention()

In [42]:
model = lstm_attn.LSTM()
model2 = lstm_attn.Embed_LSTM()

In [43]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 128, 256)          0         
_________________________________________________________________
LSTM (LSTM)                  (None, 300)               668400    
Total params: 668,400
Trainable params: 668,400
Non-trainable params: 0
_________________________________________________________________


In [37]:
def simpleNMT(pad_length=100,
              n_chars=105,
              n_labels=6,
              embedding_learnable=False,
              encoder_units=256,
              decoder_units=256,
              trainable=True,
              return_probabilities=False):
    """
    Builds a Neural Machine Translator that has alignment attention
    :param pad_length: the size of the input sequence
    :param n_chars: the number of characters in the vocabulary
    :param n_labels: the number of possible labelings for each character
    :param embedding_learnable: decides if the one hot embedding should be refinable.
    :return: keras.models.Model that can be compiled and fit'ed
    *** REFERENCES ***
    Lee, Jason, Kyunghyun Cho, and Thomas Hofmann. 
    "Neural Machine Translation By Jointly Learning To Align and Translate" 
    """
    input_ = Input(shape=(pad_length,), dtype='float32')
    input_embed = Embedding(n_chars, n_chars,
                            input_length=pad_length,
                            trainable=embedding_learnable,
                            weights=[np.eye(n_chars)],
                            name='OneHot')(input_)

    rnn_encoded = Bidirectional(LSTM(encoder_units, return_sequences=True),
                                name='bidirectional_1',
                                merge_mode='concat',
                                trainable=trainable)(input_embed)

    model = Model(inputs=input_, outputs=rnn_encoded)

    return model

In [38]:
model3 = simpleNMT()

In [39]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
OneHot (Embedding)           (None, 100, 105)          11025     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 512)          741376    
Total params: 752,401
Trainable params: 741,376
Non-trainable params: 11,025
_________________________________________________________________
